# Universal Data Cleaning & Preprocessing Playbook (Do-These-First)

Below are **12 common steps** you can apply to most tabular datasets.  
Each step includes: *what/why*, the **go-to Python function(s)**, a **template**, and then **exercises** for self-practice.

---

### How to use this section during the workshop

1. Skim each step to recall the **purpose** and the **function to use**.  
2. Run the **template** cell to see the pattern.  
3. Attempt the **exercises** right below, then compare with the instructor's solution later.

---

> **Playbook Bootstrap**  
> If `df` is not yet defined, this cell creates a small synthetic dataset so you can run the exercises immediately.

In [ ]:
try:
    assert "df" in globals() and isinstance(df, pd.DataFrame)
except Exception:
    import numpy as np, pandas as pd
    rng = np.random.default_rng(7)
    n = 500
    df = pd.DataFrame({
        "age": rng.integers(18, 70, size=n),
        "income": rng.normal(60000, 15000, size=n).round(0),
        "tenure_years": rng.exponential(3, size=n).round(2),
        "segment": rng.choice(["A","B","C"], size=n, p=[0.5,0.3,0.2]),
        "region": rng.choice(["North","South","East","West"], size=n),
        "joined_on": pd.to_datetime("2020-01-01") + pd.to_timedelta(rng.integers(0, 1500, size=n), unit="D"),
        "notes": rng.choice(["loves discounts", "calls often", "email only", "no preference"], size=n),
    })
    # inject issues
    df.loc[rng.choice(n, 20, replace=False), "income"] = np.nan
    df.loc[rng.choice(n, 10, replace=False), "age"] = None
    df.loc[rng.choice(n, 5, replace=False), "tenure_years"] = 99
    # simple target
    df["churn"] = ((df["income"].fillna(50000) < 55000) | (df["tenure_years"] < 1.5) | (df["segment"]=="C")).astype(int)
    print("Created synthetic df with shape:", df.shape)

## 1) Schema & Dtype Validation

**Why:** Wrong dtypes (numbers as strings, dates as objects) break downstream logic.  
**Go-to:** `pandas.to_numeric`, `pandas.to_datetime`, `astype`, `DataFrame.convert_dtypes()`

### Storytime: Schema & Dtype Validation  
Imagine opening a mysterious chest of data treasures. Some jewels are labeled correctly (`int`, `float`, `datetime`), but others are mislabeled—numbers stored as strings or dates written like riddles.  
**Why do this step?** Because if you don’t, downstream algorithms may stumble trying to add `"2000"` to `2000.0`.  
**Risk of skipping:** Models may silently treat numbers as text, miss calculations, or break when handling nulls. Always check the labels on your jewels before trading them.

In [ ]:
# TEMPLATE: fix dtypes
# df = df.copy()
for col in ["age", "income", "tenure_years"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df["joined_on"] = pd.to_datetime(df["joined_on"], errors="coerce")
# df = df.convert_dtypes()  # optional newer dtype inference
df.dtypes

**Exercises**
- Convert any object-numeric columns to numeric with `errors="coerce"` and report how many NaNs were created.
- Parse a date column; add `day`, `month`, `year` columns.

In [ ]:
# TODO: Your attempt here
# Example: count NaNs introduced by coercion
# before_nans = df.isna().sum()
# df["income"] = pd.to_numeric(df["income"], errors="coerce")
# after_nans = df.isna().sum()
# print(after_nans - before_nans)

<details><summary><b>Answer Key — Step 1 (Schema & Dtypes)</b></summary>

- Coerce problematic numeric strings to numbers using `pd.to_numeric(errors="coerce")` and report NaN deltas.  
- Parse datetimes and derive `day`, `month`, `year`.
</details>

In [ ]:
# Count NaNs introduced by coercion on candidate numeric cols
cand = ["age", "income", "tenure_years"]
before = df[cand].isna().sum()
for col in cand:
    df[col] = pd.to_numeric(df[col], errors="coerce")
after = df[cand].isna().sum()
print("NaNs introduced by coercion:\n", (after - before))

# Parse joined_on and add parts
df["joined_on"] = pd.to_datetime(df["joined_on"], errors="coerce")
df["day"] = df["joined_on"].dt.day
df["month"] = df["joined_on"].dt.month
df["year"] = df["joined_on"].dt.year
df[["joined_on","day","month","year"]].head()

## 2) Text Normalization (Whitespace/Case)

**Why:** Hidden spaces and case inconsistencies create artificial categories.  
**Go-to:** `str.strip`, `str.lower`, `str.replace`

### Storytime: Text Normalization  
Think of your dataset as a classroom roll call. `"North "` and `"north"` are the same student arriving with different uniforms.  
**Why do this step?** To ensure fairness—grouping all variants into one true identity.  
**Risk of skipping:** Your model believes `"North"` and `"north"` are two separate regions, splitting loyalty and weakening insights.

In [ ]:
# TEMPLATE: clean string columns
str_cols = df.select_dtypes(include="object").columns
for c in str_cols:
    df[c] = df[c].astype(str).str.strip().str.replace(r"\s+", " ", regex=True).str.lower()
df[str_cols].head()

**Exercises**
- Standardize casing in `region/segment` and validate unique values before/after.
- Remove double spaces inside any free-text column.

In [ ]:
# TODO
# uniques_before = df["region"].unique()
# df["region"] = df["region"].str.strip().str.lower()
# uniques_after = df["region"].unique()
# print(uniques_before, "->", uniques_after)

<details><summary><b>Answer Key — Step 2 (Text Normalization)</b></summary>

- Standardize whitespace and case; verify unique values changed as expected.
</details>

In [ ]:
for c in df.select_dtypes(include="object").columns:
    before_uniques = df[c].unique()
    df[c] = df[c].astype(str).str.strip().str.replace(r"\s+", " ", regex=True).str.lower()
    after_uniques = df[c].unique()
print("Normalized object columns. Example 'region' uniques:", df["region"].unique())

## 3) De-duplication

**Why:** Duplicate rows bias metrics and inflate data size.  
**Go-to:** `DataFrame.duplicated`, `drop_duplicates`

### Storytime: De-duplication  
Imagine counting villagers in a town square. If you count the same person twice, your population size grows unfairly.  
**Why do this step?** Duplicates inflate statistics, bias averages, and mislead predictions.  
**Risk of skipping:** You may design resources (or models) based on phantom citizens, leading to wasted effort or wrong conclusions.

In [ ]:
# TEMPLATE: remove duplicates, track impact
n_before = len(df)
df = df.drop_duplicates()
print("Removed", n_before - len(df), "duplicates")

**Exercises**
- Count duplicates per subset of columns (e.g., `["age","region","segment"]`).  
- Keep the **last** occurrence of duplicates and compare counts.

In [ ]:
# TODO
# dup_mask = df.duplicated(subset=["age","region","segment"], keep=False)
# df_dups = df[dup_mask]
# df_last = df.drop_duplicates(subset=["age","region","segment"], keep="last")

<details><summary><b>Answer Key — Step 3 (De-duplication)</b></summary>

- Identify duplicates by subset and compare keeping first vs last.
</details>

In [ ]:
subset = ["age","region","segment"]
dup_mask = df.duplicated(subset=subset, keep=False)
dups = df[dup_mask]
print("Duplicate groups (subset):", dups.shape[0])

keep_first = df.drop_duplicates(subset=subset, keep="first").shape[0]
keep_last  = df.drop_duplicates(subset=subset, keep="last").shape[0]
print("Rows if keep=first:", keep_first, "| keep=last:", keep_last)

## 4) Missing-Value Strategy

**Why:** Models require complete matrices; missingness also holds signal.  
**Go-to:** `SimpleImputer`, `KNNImputer`, create "Missing" category for categoricals.

### Storytime: Missing Values  
In an ancient scroll, some words have faded away. Do you ignore them, guess the missing letters, or fill them with placeholders?  
**Why do this step?** Models can’t handle blanks—they need complete sentences.  
**Risk of skipping:** Leaving gaps causes models to crash or hallucinate patterns, distorting results. The way you fill the blanks shapes the story told by the data.

In [ ]:
# TEMPLATE: numeric/categorical imputers
num_cols = df.select_dtypes(include=["number"]).columns.tolist()
cat_cols = df.select_dtypes(include=["object","category"]).columns.tolist()

from sklearn.impute import SimpleImputer, KNNImputer
num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

df[num_cols] = num_imputer.fit_transform(df[num_cols])
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])
df.isna().sum().head()

**Exercises**
- Compare `median` vs `KNNImputer` on numeric columns—measure effect on model F1.  
- Add a boolean indicator column for "was_missing" for a selected feature.

In [ ]:
# TODO
# from sklearn.metrics import f1_score
# # Fit model with median vs KNN and compare

<details><summary><b>Answer Key — Step 4 (Missing Values)</b></summary>

- Compare median vs KNN imputers by F1 on a baseline model.  
- Add missingness indicator.
</details>

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer

X = df.drop(columns=["churn"])
y = df["churn"].astype(int)

num_cols = X.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object","category"]).columns.tolist()

# Add a missingness indicator for income
X["income_was_missing"] = X["income"].isna().astype(int)

pre_median = ColumnTransformer([
    ("num", Pipeline([("imp", SimpleImputer(strategy="median")), ("sc", StandardScaler())]), num_cols),
    ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")), ("oh", OneHotEncoder(handle_unknown="ignore"))]), cat_cols)
])

pre_knn = ColumnTransformer([
    ("num", Pipeline([("imp", KNNImputer(n_neighbors=5)), ("sc", StandardScaler())]), num_cols),
    ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")), ("oh", OneHotEncoder(handle_unknown="ignore"))]), cat_cols)
])

for name, pre in [("median", pre_median), ("knn", pre_knn)]:
    pipe = Pipeline([("pre", pre), ("clf", LogisticRegression(max_iter=300))])
    scores = cross_val_score(pipe, X, y, cv=5, scoring="f1")
    print(f"{name} imputer CV F1 mean:", scores.mean().round(3))

## 5) Outlier Handling

**Why:** Extremes can distort means and model coefficients.  
**Go-to:** Quantile capping (`clip`), `LocalOutlierFactor` (inspection), `IsolationForest`.

### Storytime: Outliers  
Picture a feast where one guest eats 100 plates while everyone else eats 3. That single guest distorts the banquet’s average.  
**Why do this step?** Outliers can dominate scales, shift means, and mislead models.  
**Risk of skipping:** Your model may believe everyone is a glutton, overestimating costs or risks. Handling outliers keeps the feast realistic.

In [ ]:
# TEMPLATE: winsorize/cap and flag with LOF
cap = df["tenure_years"].quantile(0.99)
df["tenure_years"] = df["tenure_years"].clip(upper=cap)

from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.02)
is_inlier = lof.fit_predict(df[num_cols]) == 1
print("Flagged outliers:", (~is_inlier).sum())

**Exercises**
- Try 95th vs 99th percentile caps and visualize histograms side-by-side.  
- Drop LOF-flagged outliers and compare CV metrics.

In [ ]:
# TODO
# df95 = df.copy()
# df95["tenure_years"] = df95["tenure_years"].clip(upper=df95["tenure_years"].quantile(0.95))
# ax = df["tenure_years"].hist(bins=30)
# ax = df95["tenure_years"].hist(bins=30, alpha=0.5)

<details><summary><b>Answer Key — Step 5 (Outliers)</b></summary>

- Compare caps at different percentiles visually and quantify effect.
</details>

In [ ]:
import matplotlib.pyplot as plt
df95 = df.copy()
df99 = df.copy()
df95["tenure_years"] = df95["tenure_years"].clip(upper=df95["tenure_years"].quantile(0.95))
df99["tenure_years"] = df99["tenure_years"].clip(upper=df99["tenure_years"].quantile(0.99))

df["tenure_years"].hist(bins=30); plt.title("Original tenure_years"); plt.show()
df95["tenure_years"].hist(bins=30); plt.title("Capped at 95th"); plt.show()
df99["tenure_years"].hist(bins=30); plt.title("Capped at 99th"); plt.show()

## 6) Business-Rule Validation

**Why:** Guardrails against bad upstream data.  
**Go-to:** `assert`, bounded checks, membership tests.

### Storytime: Business Rules  
Imagine a guard at the city gate checking travelers: no child under 18 may enter the tavern.  
**Why do this step?** Business rules enforce domain reality—ages, ranges, memberships.  
**Risk of skipping:** A 5-year-old might be counted as a loan applicant or a negative income might sneak into salary records, shattering trust and logic.

In [ ]:
# TEMPLATE: assertions
assert (df["age"] >= 0).all(), "Age must be non-negative"
assert df["segment"].isin(["a","b","c"]).all(), "Unexpected segment values"
print("Business rules passed.")

**Exercises**
- Add a rule for `income` minimum; fail fast if violated.  
- Create a function `validate_schema(df)` that returns a dict of failures.

In [ ]:
# TODO
# def validate_schema(df):
#     errors = {}
#     if not (df["income"] >= 0).all():
#         errors["income"] = "negative values found"
#     return errors

<details><summary><b>Answer Key — Step 6 (Business Rules)</b></summary>

- Validate ranges; return a dictionary of failures.
</details>

In [ ]:
def validate_schema(df):
    errors = {}
    if not (df["income"].fillna(0) >= 0).all():
        errors["income"] = "negative or invalid values"
    if not df["segment"].isin(["a","b","c"]).all():
        errors["segment"] = "unexpected categories present"
    return errors

validate_schema(df)

## 7) Categorical Encoding

**Why:** Models expect numeric inputs.  
**Go-to:** `OneHotEncoder` for nominal; `OrdinalEncoder` for ordered categories.

### Storytime: Categorical Encoding  
Think of categories as different languages—French, Hindi, Japanese. Models only understand numbers.  
**Why do this step?** Encoding translates human-friendly categories into model-friendly digits.  
**Risk of skipping:** The model hears gibberish—strings it cannot process. Worse, wrong encoding (like ordinals where none exist) tells the model a false story of rank.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

cat_cols = ["region","segment"] if set(["region","segment"]).issubset(df.columns) else df.select_dtypes(include="object").columns.tolist()

cat_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(drop="first", handle_unknown="ignore"))
])

**Exercises**
- Use `OrdinalEncoder` with an explicit order for `segment` and compare model performance vs one-hot.  
- Measure dimensionality growth after one-hot.

In [ ]:
# TODO
# ord = OrdinalEncoder(categories=[["c","b","a"]])
# df["segment_ord"] = ord.fit_transform(df[["segment"]])

<details><summary><b>Answer Key — Step 7 (Categorical Encoding)</b></summary>

- Ordinal-encode `segment` with explicit order and inspect distribution.
</details>

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
if "segment" in df.columns:
    ord = OrdinalEncoder(categories=[["c","b","a"]])
    df["segment_ord"] = ord.fit_transform(df[["segment"]])
    print(df["segment_ord"].value_counts().sort_index())

## 8) Scaling Numerical Features

**Why:** Many models are sensitive to feature scales.  
**Go-to:** `StandardScaler`, `MinMaxScaler`, `RobustScaler`.

### Storytime: Scaling  
Imagine runners in a race: one measured in meters, another in kilometers. Who seems faster?  
**Why do this step?** Scaling puts all features on comparable scales, ensuring fair competition.  
**Risk of skipping:** Models that rely on distances (kNN, SVM, PCA) will crown the wrong champion, misjudging importance.

In [ ]:
from sklearn.preprocessing import StandardScaler
num_cols = df.select_dtypes(include=["number"]).columns.tolist()
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[num_cols] = scaler.fit_transform(df_scaled[num_cols])
df_scaled[num_cols].head()

**Exercises**
- Compare `StandardScaler` vs `RobustScaler` on outlier-heavy data.  
- Show effect on a k-NN classifier’s performance.

In [ ]:
# TODO
# from sklearn.preprocessing import RobustScaler
# scaler2 = RobustScaler()
# df_scaled2 = df.copy()
# df_scaled2[num_cols] = scaler2.fit_transform(df_scaled2[num_cols])

<details><summary><b>Answer Key — Step 8 (Scaling)</b></summary>

- Compare Standard vs Robust scaling effects.
</details>

In [ ]:
from sklearn.preprocessing import RobustScaler, StandardScaler
num_cols = df.select_dtypes(include=["number"]).columns.tolist()
ss = StandardScaler().fit_transform(df[num_cols])
rs = RobustScaler().fit_transform(df[num_cols])
print("StdScaler mean (approx 0):", ss.mean().round(4), " | Robust median ~0 (not printed)")

## 9) Datetime Feature Engineering

**Why:** Raw datetimes need transformation to become useful.  
**Go-to:** `.dt.year`, `.dt.month`, timedeltas.

### Storytime: Datetime Features  
Your dataset has birthdays, but the real magic is in age.  
**Why do this step?** Raw timestamps hide patterns; extracting years, months, and durations uncovers trends like seasonality and recency.  
**Risk of skipping:** The model stares at raw dates with no clue how to compare `"2022-03-01"` to `"2021-09-15"`, losing temporal wisdom.

In [ ]:
dt = pd.to_datetime(df["joined_on"], errors="coerce")
df["join_year"] = dt.dt.year
df["join_month"] = dt.dt.month
df["tenure_days"] = (pd.Timestamp.today().normalize() - dt).dt.days
df[["join_year","join_month","tenure_days"]].head()

**Exercises**
- Create `is_weekend`, `quarter`, and `days_since_year_start`.  
- Bucketize `tenure_days` into 5 quantile bins.

In [ ]:
# TODO
# df["quarter"] = dt.dt.quarter
# df["is_weekend"] = dt.dt.weekday >= 5

<details><summary><b>Answer Key — Step 9 (Datetime)</b></summary>

- Weekend and quarter flags; days since year start.
</details>

In [ ]:
dt = pd.to_datetime(df["joined_on"], errors="coerce")
df["quarter"] = dt.dt.quarter
df["is_weekend"] = (dt.dt.weekday >= 5).astype(int)
df["days_since_year_start"] = (dt - pd.to_datetime(dt.dt.year.astype(str) + "-01-01")).dt.days
df[["quarter","is_weekend","days_since_year_start"]].head()

## 10) Binning & Threshold Features

**Why:** Capture non-linear thresholds with simpler models.  
**Go-to:** `KBinsDiscretizer`, `pd.cut`, `pd.qcut`.

### Storytime: Binning  
Think of exam scores grouped into grades A, B, C. Small differences vanish, but broad categories reveal clearer groups.  
**Why do this step?** Binning captures thresholds and simplifies non-linear relationships.  
**Risk of skipping:** Subtle patterns stay hidden; the model may struggle with raw continuous values where thresholds matter.

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
kb = KBinsDiscretizer(n_bins=4, encode="ordinal", strategy="quantile")
df["income_bin"] = kb.fit_transform(df[["income"]]).astype(int)
df["age_bin"] = kb.fit_transform(df[["age"]]).astype(int)
df[["income","income_bin","age","age_bin"]].head()

**Exercises**
- Compare `pd.cut` vs `pd.qcut` for `income`; inspect bin sizes.  
- Train a simple logistic regression on binned vs raw features.

In [ ]:
# TODO
# df["income_qcut"] = pd.qcut(df["income"], q=4, duplicates="drop")
# df["income_cut"] = pd.cut(df["income"], bins=4)

<details><summary><b>Answer Key — Step 10 (Binning)</b></summary>

- Compare equal-width vs quantile bins.
</details>

In [ ]:
df["income_qcut"] = pd.qcut(df["income"], q=4, duplicates="drop")
df["income_cut"] = pd.cut(df["income"], bins=4)
print("qcut sizes:\n", df["income_qcut"].value_counts().sort_index())
print("cut sizes:\n", df["income_cut"].value_counts().sort_index())

## 11) Feature Selection (Filter/Wrappers)

**Why:** Reduce noise, speed up training, improve generalization.  
**Go-to:** `VarianceThreshold`, `SelectKBest(mutual_info_)`, model-based (`RFECV`, `SelectFromModel`).

### Storytime: Feature Selection  
Picture a storyteller with 100 characters. Too many side plots distract from the hero’s journey.  
**Why do this step?** Feature selection removes noisy, irrelevant features, sharpening focus.  
**Risk of skipping:** The model wastes time on background noise, overfits, and forgets the main storyline.

In [ ]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_classif
X = df.drop(columns=[c for c in ["churn"] if c in df.columns])
y = df["churn"].astype(int) if "churn" in df.columns else (df.iloc[:, -1] > df.iloc[:, -1].median()).astype(int)

vt = VarianceThreshold(0.0)
X_vt = vt.fit_transform(X.select_dtypes(include=["number"]))
skb = SelectKBest(score_func=mutual_info_classif, k=min(10, X_vt.shape[1]))
X_sel = skb.fit_transform(X_vt, y)
print("Selected features:", skb.get_support(indices=True))

**Exercises**
- Plot top-10 MI scores as a bar chart.  
- Try `RFECV` with logistic regression.

In [ ]:
# TODO
# from sklearn.feature_selection import RFECV
# from sklearn.linear_model import LogisticRegression
# rfecv = RFECV(LogisticRegression(max_iter=300), cv=5, scoring="f1")
# rfecv.fit(X.select_dtypes(include=["number"]), y)

<details><summary><b>Answer Key — Step 11 (Feature Selection)</b></summary>

- Plot MI scores and run RFECV.
</details>

In [ ]:
from sklearn.feature_selection import mutual_info_classif, RFECV
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt

numX = df.select_dtypes(include=["number"]).drop(columns=["churn"], errors="ignore")
y = df["churn"].astype(int) if "churn" in df.columns else (df.iloc[:, -1] > df.iloc[:, -1].median()).astype(int)
mi = mutual_info_classif(numX.fillna(numX.median()), y, random_state=42)
order = np.argsort(mi)[::-1]
plt.bar(range(min(10, len(order))), mi[order][:10])
plt.xticks(range(min(10, len(order))), numX.columns[order][:10], rotation=45, ha="right")
plt.title("Top MI Features"); plt.tight_layout(); plt.show()

rfecv = RFECV(LogisticRegression(max_iter=300), cv=5, scoring="f1")
rfecv.fit(numX.fillna(numX.median()), y)
print("Optimal features:", rfecv.n_features_)

## 12) Leakage-Safe Split & Pipelines

**Why:** Prevent target leakage by fitting transforms only on training folds.  
**Go-to:** `train_test_split`, `Pipeline`, `ColumnTransformer`, `cross_val_score`.

### Storytime: Leakage-Safe Pipelines  
Imagine training a detective using tomorrow’s newspaper. He solves cases perfectly—but cheats by knowing the future.  
**Why do this step?** Pipelines ensure preprocessing fits only on training folds, not leaking test knowledge.  
**Risk of skipping:** Models score brilliantly in practice but fail miserably in the real world.

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression

features = df.drop(columns=[c for c in ["churn"] if c in df.columns])
target = df["churn"].astype(int) if "churn" in df.columns else (df.iloc[:, -1] > df.iloc[:, -1].median()).astype(int)

num_cols = features.select_dtypes(include=["number"]).columns.tolist()
cat_cols = features.select_dtypes(include=["object","category"]).columns.tolist()

num_pipe = Pipeline([("impute", SimpleImputer(strategy="median")), ("scale", StandardScaler())])
cat_pipe = Pipeline([("impute", SimpleImputer(strategy="most_frequent")), ("onehot", OneHotEncoder(drop="first", handle_unknown="ignore"))])

pre = ColumnTransformer([("num", num_pipe, num_cols), ("cat", cat_pipe, cat_cols)])

pipe = Pipeline([("pre", pre), ("select", SelectKBest(mutual_info_classif, k=min(12, len(num_cols)+len(cat_cols)))), ("clf", LogisticRegression(max_iter=300))])

X_tr, X_te, y_tr, y_te = train_test_split(features, target, stratify=target, test_size=0.25, random_state=42)
cv = cross_val_score(pipe, X_tr, y_tr, cv=5, scoring="f1")
print("CV F1:", cv.mean().round(3))

**Exercises**
- Add `KBinsDiscretizer` into the numeric branch of the pipeline and evaluate.  
- Swap `LogisticRegression` with `RandomForestClassifier` and compare F1.

In [ ]:
# TODO
# from sklearn.ensemble import RandomForestClassifier
# pipe2 = Pipeline([("pre", pre), ("clf", RandomForestClassifier(random_state=42))])
# scores = cross_val_score(pipe2, X_tr, y_tr, cv=5, scoring="f1")
# print(scores.mean())

<details><summary><b>Answer Key — Step 12 (Pipelines)</b></summary>

- Add KBins to numeric branch and compare RF baseline.
</details>

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

features = df.drop(columns=["churn"])
target = df["churn"].astype(int)

num_cols = features.select_dtypes(include=["number"]).columns.tolist()
cat_cols = features.select_dtypes(include=["object","category"]).columns.tolist()

num_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("bins", KBinsDiscretizer(n_bins=5, encode="ordinal", strategy="quantile")),
])

cat_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("oh", OneHotEncoder(handle_unknown="ignore"))
])

pre = ColumnTransformer([("num", num_pipe, num_cols), ("cat", cat_pipe, cat_cols)])

rf = Pipeline([("pre", pre), ("rf", RandomForestClassifier(n_estimators=200, random_state=42))])
scores = cross_val_score(rf, features, target, cv=5, scoring="f1")
print("RF with KBins CV F1:", scores.mean().round(3))

---
# (Original walkthrough below)

# Data Processing & Feature Engineering (3-Hour Hands‑On)

**Audience:** Early professionals  
**Focus:** Cleaning → Transformation → Feature Extraction  
**Instructor Pack:** Each section has runnable code, notes, and mini‑exercises.

## Agenda (3 Hours)

- **0:00–0:10 (10 min)** — Kickoff & goals  
- **0:10–0:35 (25 min)** — Data ingestion & quick EDA (tabular focus; touch text/datetime)  
- **0:35–1:20 (45 min)** — Data cleaning (missing values, outliers, duplicates, schema)  
- **1:20–2:05 (45 min)** — Transformations (scaling, encoding, binning, datetime, text basics)  
- **2:05–2:40 (35 min)** — Feature extraction & selection (PCA, mutual info, filters)  
- **2:40–3:00 (20 min)** — End‑to‑end Pipeline + leakage checks + Q&A

## 0) Environment Setup

> Tip: Run this once at the start of the session.

In [ ]:
# Core stack
import numpy as np
import pandas as pd

# Viz
import matplotlib.pyplot as plt

# Sklearn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif, VarianceThreshold, SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import LocalOutlierFactor

# Misc
pd.set_option("display.max_columns", 100)

**What we achieve here**
- Import NumPy/Pandas for data handling, Matplotlib for quick charts.
- Bring in core scikit‑learn tools (imputers, encoders, scalers, pipelines, feature selectors, PCA, model, metrics).
- Set Pandas display options so participants can see wide tables without truncation.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- **SimpleImputer** replaces missing values with median (numeric) or most frequent (categorical).


## 1) Synthetic, Realistic Dataset

We generate a small dataset with numeric, categorical, datetime, and a tiny text field—plus intentional issues: missing values, outliers, and duplicates.

### Storytime: Schema & Dtype Validation  
Imagine opening a mysterious chest of data treasures. Some jewels are labeled correctly (`int`, `float`, `datetime`), but others are mislabeled—numbers stored as strings or dates written like riddles.  
**Why do this step?** Because if you don’t, downstream algorithms may stumble trying to add `"2000"` to `2000.0`.  
**Risk of skipping:** Models may silently treat numbers as text, miss calculations, or break when handling nulls. Always check the labels on your jewels before trading them.

In [ ]:
rng = np.random.default_rng(7)
n = 500

df = pd.DataFrame({
    "age": rng.integers(18, 70, size=n),
    "income": rng.normal(60000, 15000, size=n).round(0),
    "tenure_years": rng.exponential(3, size=n).round(2),
    "segment": rng.choice(["A","B","C"], size=n, p=[0.5,0.3,0.2]),
    "region": rng.choice(["North","South","East","West"], size=n),
    "joined_on": pd.to_datetime("2020-01-01") + pd.to_timedelta(rng.integers(0, 1500, size=n), unit="D"),
    "notes": rng.choice(["loves discounts", "calls often", "email only", "no preference"], size=n),
})

# introduce some issues
df.loc[rng.choice(n, 20, replace=False), "income"] = np.nan              # missing
df.loc[rng.choice(n, 10, replace=False), "age"] = None                   # missing
df.loc[rng.choice(n, 5, replace=False), "tenure_years"] = 99             # outliers
dups = df.sample(5, random_state=1)
df = pd.concat([df, dups], ignore_index=True)                            # duplicates

# Binary target with a non-trivial relationship
df["churn"] = ((df["income"].fillna(50000) < 55000) | (df["tenure_years"] < 1.5) | (df["segment"]=="C")).astype(int)

df.head()

**What we achieve here**
- Build a **synthetic but realistic** dataset with mixed types: numeric, categorical, datetime, and short text.
- Intentionally inject **missing values**, **outliers**, and **duplicates** so we can practice cleaning.
- Create a **target (`churn`)** with a non‑trivial relationship to features to validate feature engineering later.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- This cell applies transformations/analysis to progress through data cleaning or feature engineering.


## 2) Quick EDA (10–25 min)

Check shape, schema, missingness, class balance, and distributions. Spot potential leakage and sanity‑check duplicates/outliers.

### Storytime: Text Normalization  
Think of your dataset as a classroom roll call. `"North "` and `"north"` are the same student arriving with different uniforms.  
**Why do this step?** To ensure fairness—grouping all variants into one true identity.  
**Risk of skipping:** Your model believes `"North"` and `"north"` are two separate regions, splitting loyalty and weakening insights.

In [ ]:
print(df.shape, df.dtypes)
display(df.describe(include="all").T.head(12))
display(df.isna().mean().sort_values(ascending=False))
display(df["segment"].value_counts(normalize=True))
ax = df["age"].plot(kind="hist", bins=20)
ax.set_title("Age Distribution")
plt.show()
display(pd.crosstab(df["segment"], df["churn"], normalize="index"))

**What we achieve here**
- Quick **shape & schema** check to confirm row/column counts and dtypes.
- `describe()` summarizes distributions; `isna().mean()` highlights **missingness** by column.
- Histogram shows **distribution & skew**; crosstab inspects **class balance by segment**.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- **EDA functions** summarize schema, missingness, distributions, and target correlation.


## 3) Data Cleaning (35–80 min)

We handle duplicates, imputations, outliers, and schema rules.

### Storytime: De-duplication  
Imagine counting villagers in a town square. If you count the same person twice, your population size grows unfairly.  
**Why do this step?** Duplicates inflate statistics, bias averages, and mislead predictions.  
**Risk of skipping:** You may design resources (or models) based on phantom citizens, leading to wasted effort or wrong conclusions.

In [ ]:
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print(f"Removed {before-after} duplicate rows.")

**What we achieve here**
- Remove exact duplicate rows to prevent **data leakage** and **biased metrics**.
- The before/after print quantifies the clean‑up.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- **drop_duplicates()** removes duplicate rows from the dataset.


In [ ]:
num_cols = ["age", "income", "tenure_years"]
cat_cols = ["segment", "region"]

num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

df[num_cols] = num_imputer.fit_transform(df[num_cols])
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])

df.isna().sum()

**What we achieve here**
- Impute **numeric** features with median (robust to outliers) and **categoricals** with most‑frequent to keep valid categories.
- Ensures downstream models receive **complete matrices** without NaNs.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- **SimpleImputer** replaces missing values with median (numeric) or most frequent (categorical).


In [ ]:
# Cap extreme tenure values at 95th percentile
cap = df["tenure_years"].quantile(0.95)
df["tenure_years"] = np.where(df["tenure_years"] > cap, cap, df["tenure_years"])

# Unsupervised detection (inspect, then decide)
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.02)
mask_inlier = lof.fit_predict(df[num_cols]) == 1
print("Outliers flagged by LOF:", (~mask_inlier).sum())
# Optionally filter:
# df = df[mask_inlier]

**What we achieve here**
- **Cap** extreme values to reduce the influence of outliers (winsorization).
- Use **LocalOutlierFactor** to *flag* unusual rows; teams can decide to drop or keep after inspection.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- **LocalOutlierFactor** flags outliers based on neighborhood density differences.


In [ ]:
# Basic schema checks
assert (df["age"] >= 0).all(), "Age must be non-negative"
assert df["segment"].isin(["A","B","C"]).all()
print("Schema checks passed.")

**What we achieve here**
- Enforce **business rules** (age non‑negative, segment in {A,B,C}).
- Early assertions catch schema drifts or bad upstream data before modeling.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- This cell applies transformations/analysis to progress through data cleaning or feature engineering.


## 4) Transformations (80–125 min)

Scaling, encoding, binning, datetime expansion, and tiny text features.

### Storytime: Missing Values  
In an ancient scroll, some words have faded away. Do you ignore them, guess the missing letters, or fill them with placeholders?  
**Why do this step?** Models can’t handle blanks—they need complete sentences.  
**Risk of skipping:** Leaving gaps causes models to crash or hallucinate patterns, distorting results. The way you fill the blanks shapes the story told by the data.

In [ ]:
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[num_cols] = scaler.fit_transform(df_scaled[num_cols])
df_scaled[num_cols].head()

**What we achieve here**
- **Standardize** numeric columns (zero mean, unit variance) so linear models and distance‑based methods behave well.
- Store results in a copy (`df_scaled`) to keep a clean audit trail.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- **StandardScaler** scales numeric features to mean=0 and variance=1.


In [ ]:
# One-Hot for nominal 'region'
ohe = OneHotEncoder(drop="first", sparse_output=False, handle_unknown="ignore")
region_ohe = ohe.fit_transform(df_scaled[["region"]])
region_ohe = pd.DataFrame(region_ohe, columns=ohe.get_feature_names_out(["region"]))
df_enc = pd.concat([df_scaled.drop(columns=["region"]), region_ohe], axis=1)

# Ordinal for ordered 'segment' (assume C < B < A)
ord_enc = OrdinalEncoder(categories=[["C","B","A"]])
df_enc["segment_ord"] = ord_enc.fit_transform(df[["segment"]])
df_enc.head()

**What we achieve here**
- Apply **One‑Hot Encoding** to nominal `region` (drop one column to avoid dummy trap).
- Apply **Ordinal Encoding** to `segment` where an order is assumed (C < B < A) for models that can use ordinality.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- **OneHotEncoder** converts categorical variables into binary indicator columns.


In [ ]:
kb = KBinsDiscretizer(n_bins=4, encode="ordinal", strategy="quantile")
df_enc["income_bin"] = kb.fit_transform(df[["income"]]).astype(int)
df_enc["age_bin"] = kb.fit_transform(df[["age"]]).astype(int)

**What we achieve here**
- **Discretize** continuous variables into bins to capture **non‑linear** effects and allow tree/linear models to exploit thresholds.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- **KBinsDiscretizer** bins continuous values into discrete intervals.


In [ ]:
dt = pd.to_datetime(df["joined_on"])
df_enc["join_year"]  = dt.dt.year
df_enc["join_month"] = dt.dt.month
df_enc["tenure_days"] = (pd.Timestamp.today().normalize() - dt).dt.days

**What we achieve here**
- Engineer **calendar** features (year, month) and **tenure in days** to turn datetime into useful numeric signals.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- **Datetime accessor (.dt)** extracts parts like year, month, and computes durations.


In [ ]:
df_enc["notes_len"] = df["notes"].str.len()
for kw in ["discount", "email", "call"]:
    df_enc[f"kw_{kw}"] = df["notes"].str.contains(kw, case=False).astype(int)
df_enc.head()

**What we achieve here**
- Create lightweight **text features** without full NLP: length and keyword presence flags (`discount`, `email`, `call`).
- Great for tabular problems where text is short and domain‑specific.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- **str.contains** checks if keywords exist in text, creating binary features.


## 5) Feature Extraction & Selection (125–160 min)

Variance filtering, Mutual Information, and PCA (unsupervised on numeric block).

### Storytime: Outliers  
Picture a feast where one guest eats 100 plates while everyone else eats 3. That single guest distorts the banquet’s average.  
**Why do this step?** Outliers can dominate scales, shift means, and mislead models.  
**Risk of skipping:** Your model may believe everyone is a glutton, overestimating costs or risks. Handling outliers keeps the feast realistic.

In [ ]:
X0 = df_enc.drop(columns=["churn", "notes", "joined_on", "segment"])
y  = df["churn"].astype(int)

vt = VarianceThreshold(threshold=0.0)
X_vt = vt.fit_transform(X0)
X_vt = pd.DataFrame(X_vt, columns=X0.columns[vt.get_support()])
X_vt.shape

**What we achieve here**
- Remove **near-constant** features that carry little to no information and can slow training.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- **VarianceThreshold** removes features with near-constant variance.


In [ ]:
mi = mutual_info_classif(X_vt, y, random_state=42)
mi_series = pd.Series(mi, index=X_vt.columns).sort_values(ascending=False)
topk = 10
display(mi_series.head(topk))

**What we achieve here**
- Use **Mutual Information** (non‑parametric) to score feature–target dependency and pick top‑k informative features.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- **mutual_info_classif** scores features by dependency with the target variable.


In [ ]:
selector = SelectKBest(score_func=mutual_info_classif, k=topk)
X_mi = selector.fit_transform(X_vt, y)
selected_cols = X_vt.columns[selector.get_support()]
selected_cols

**What we achieve here**
- **SelectKBest** keeps the k highest‑scoring features (here via MI), reducing dimensionality/noise before modeling.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- **mutual_info_classif** scores features by dependency with the target variable.


In [ ]:
num_cols_only = ["age", "income", "tenure_years"]
num_block = df_enc[num_cols_only].to_numpy()
pca = PCA(n_components=2, random_state=42)
num_pcs = pca.fit_transform(num_block)
print("Explained variance (2 PCs):", pca.explained_variance_ratio_.sum().round(3))

**What we achieve here**
- Run **PCA** on numeric block to create low‑dimensional components capturing maximum variance; useful for compression and visualization.
- We report explained variance to show how much signal the 2 PCs retain.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- **PCA** reduces dimensionality by projecting onto principal components while preserving variance.


## 6) End‑to‑End Pipeline (160–180 min)

A leakage‑safe pipeline: imputation, scaling/encoding, selection, and model—**inside** CV folds.

### Storytime: Business Rules  
Imagine a guard at the city gate checking travelers: no child under 18 may enter the tavern.  
**Why do this step?** Business rules enforce domain reality—ages, ranges, memberships.  
**Risk of skipping:** A 5-year-old might be counted as a loan applicant or a negative income might sneak into salary records, shattering trust and logic.

In [ ]:
base = df.drop(columns=["notes", "joined_on"])  # drop raw text/datetime for this demo
X = base.drop(columns=["churn"])
y = base["churn"].astype(int)

num_cols_pipe = X.select_dtypes(include=["int64","float64"]).columns.tolist()
cat_cols_pipe = X.select_dtypes(include=["object"]).columns.tolist()

numeric_pipe = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler())
])

categorical_pipe = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(drop="first", handle_unknown="ignore"))
])

pre = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols_pipe),
        ("cat", categorical_pipe, cat_cols_pipe),
    ]
)

model_pipe = Pipeline(steps=[
    ("pre", pre),
    ("varth", VarianceThreshold(0.0)),
    ("select", SelectKBest(mutual_info_classif, k=12)),
    ("clf", LogisticRegression(max_iter=200))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

cv_scores = cross_val_score(model_pipe, X_train, y_train, cv=5, scoring="f1")
print("CV F1:", cv_scores.round(3), "mean:", cv_scores.mean().round(3))

model_pipe.fit(X_train, y_train)
pred = model_pipe.predict(X_test)
print(classification_report(y_test, pred))

**What we achieve here**
- Build a **leakage‑safe** end‑to‑end `Pipeline`:
  - `ColumnTransformer` applies numeric/categorical preprocessing **inside CV folds**.
  - `VarianceThreshold` and `SelectKBest` run **after** preprocessing, still inside CV.
  - `LogisticRegression` is the baseline model.
- Evaluate with **cross‑validated F1** and report a clean **classification report** on held‑out test.

### Explanation
This block demonstrates how the function(s) above work and what we achieve.
- **SimpleImputer** replaces missing values with median (numeric) or most frequent (categorical).


## Mini‑Exercises

1. **Imputer swap:** Use `KNNImputer` for numerics; compare CV F1.  
2. **Encoding change:** Use `OrdinalEncoder` for `segment` instead of One‑Hot; compare.  
3. **Binning:** Add `KBinsDiscretizer` for `income` within the numeric pipeline.  
4. **PCA in pipeline:** Insert a small PCA on numeric features before selection; observe impact.  
5. **Outliers:** Filter 2% LOF outliers prior to training; discuss pros/cons.

## Common Pitfalls — Checklist

- Leakage by fitting scalers/encoders/selectors **before** split/CV.  
- Treating datetimes as raw numbers; extract parts or durations instead.  
- High‑cardinality one‑hot blow‑ups (consider hashing or target encoding—use with care).  
- Imbalanced targets → track precision/recall/F1, not just accuracy.  
- Over‑engineered features that don’t generalize—validate with CV.

## Optional Add‑Ons (Homework / Bonus)

- **Text features:** Try `TfidfVectorizer` on `notes` via `ColumnTransformer`.  
- **Leakage demo:** Add an obvious post‑event variable and see inflated CV.  
- **Drift checks:** Compare train/test distributions (e.g., KS test, PSI).